# 1. 使用FUNCTION_CALL模式

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.schema import prompt_template
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)



# 提供提示词模版(以ChatPromptTemplate为例)
prompt_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手,根据用户的提问,必要时调用Search工具,使用互联网检索数据"),
    ("human","{input}"),
    ("system","{agent_scratchpad}"),
    ("system","{chat_history}"),
    
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", # 此值需要与提示词模版中的记录记忆的变量同名
)

# 获取Agent的实例: create_tool_calling
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)






> Entering new AgentExecutor chain...

Invoking: `Search` with `汕尾天气预报`


[{'title': '汕尾 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59501', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 24.2℃ | 26.2℃ | 29.4℃ | 29.8℃ | 28.4℃ | 27.8℃ | 25.4℃ | 22.6℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm |\n| 风速 | 7.9m/s | 7.9m/s | 5.6m/s | 5.1m/s | 5.2m/s | 7.6m/s | 9.4m/s | 9.3m/s |\n| 风向 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 |\n| 气压 | 1008.5hPa | 1007.1hPa | 1005.8hPa | 1006.5hPa | 1007.3hPa | 1007.4hPa | 1007.5hPa | 1008.2hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 20.2℃ | 21.8℃ | 23.7℃ | 23.8℃ | 23.7℃ | 21.8℃ | 21℃ | 20.2℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.5m/s | 7.7m/s | 7.5m/s | 7.9m/s | 7.5m/s

In [4]:
result =  agent_exector.invoke({"input":"上海的呢？"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `Search` with `上海今天的天气情况`


[{'title': '中国气象局-天气预报-城市预报- 上海', 'url': 'https://weather.cma.cn/web/weather/58367.html', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 05:00 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 |\n| 天气 |\n| 气温 | 23℃ | 24.9℃ | 26.4℃ | 28.8℃ | 27℃ | 26.5℃ | 27.3℃ | 24.7℃ |\n| 降水 | 0.2mm | 0.3mm | 0.1mm | 0.3mm | 0.2mm | 0.2mm | 无降水 | 0.2mm |\n| 风速 | 3.3m/s | 3.2m/s | 3.2m/s | 3.3m/s | 2.9m/s | 2.7m/s | 2.9m/s | 3m/s |\n| 风向 | 东北风 | 东南风 | 东南风 | 东南风 | 西南风 | 西南风 | 西北风 | 东南风 |\n| 气压 | 1015.7hPa | 1016.6hPa | 1015.8hPa | 1014.1hPa | 1014.3hPa | 1015.7hPa | 1015.8hPa | 1015.8hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 24.9℃ | 26.4℃ | 28.8℃ | 27℃ | 26.5℃ | 27.3℃ | 24.7℃ | 23.7℃ |\n| 降水 | 0.3mm | 0.1mm | 0.3mm | 0.2mm | 0.2mm | 无降水 | 0.2mm | 1.2mm |\n| 风速 | 3.2m/s | 3.2m/s | 3.3m/s | 

# 2. 使用ReACT模式

## 举例1: 使用网络上现成的提示词模版结构

In [5]:
from langchain import hub
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)



prompt_template =  hub.pull("hwchase17/react-chat")

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", 
)

# 提供

# 获取Agent的实例: create_tool_calling
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)




C:\Users\Admin\.conda\envs\py310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "汕尾天气情况 2023年10月23日"
```[{'title': '23到25日平稳回温早晚清凉 - 广东- 中国天气网', 'url': 'http://gd.weather.com.cn/syxw/10/3661033.shtml', 'content': ':   曼谷 东京 首尔 吉隆坡 新加坡 巴黎 罗马 伦敦 雅典 柏林 纽约 温哥华 墨西哥城 哈瓦那 圣何塞 巴西利亚 布宜诺斯艾利斯 圣地亚哥 利马 基多 悉尼 墨尔本 惠灵顿 奥克兰 苏瓦 开罗 内罗毕 开普敦 维多利亚 拉巴特\n\n选择洲际\n\n:   亚洲 欧洲 北美洲 南美洲 非洲 大洋洲\n\n23到25日平稳回温 早晚清凉\n\n### 【字体：大;) 中;) 小;)】 2023-10-23 11:27:22   来源： 中国天气网广东站\n\n预计，23到25日我省大部多云为主，各地气温继续回升，早晚天气清凉，部分市县有（轻）雾。\n\n天气实况：22日我省大部阴天为主，出现了小雨，湛江西北部、茂名西部出现了局部中雨，湛江市廉江市塘莲镇录得单点的大雨25.5毫米；白天我省最高气温普遍上升，其中汕尾惠来录得全省最高气温28.6℃。\n\n具体预报：23日, 韶关、清远、肇庆、云浮晴间多云，粤东多云为主，其余市县多云间阴天，局部有零星小雨。\n\n24日，我省大部多云间晴天气为主，局部有零星小雨，其中受偏东气流影响，雷州半岛有中雨。最低气温：中北部市县18～21℃，南部市县21～23℃。', 'score': 0.89567816}, {'title': '汕尾2023年10月历史天气查询', 'url': 'https://www.zgjia.com/lishi/shanwei/202310.html', 'content': '汕尾2023年10月天气概述  广东省汕尾市2023年10月天气信息如下：整月白天平均温度为28.6度，夜间平均温度为23度，最高温度是发生在2023年10月05日的34度，最低温度是2023年10月08日的26摄氏度，本月天气总体以多云为

In [6]:
result = agent_exector.invoke({"input":"上海的呢?"})

print(result)




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "上海天气预报 2023年10月23日"
```[{'title': '上海历史天气预报2023年10月份', 'url': 'https://www.tianqihoubao.com/lishi/shanghai/month/202310.html', 'content': '前一月  后一月  上海历史天气  上海10月份天气统计信息 搜索"城市名+天气后报"访问本站（上海天气后报） 导出Excel\n\n### 天气状况统计\n\n2023年10月份上海上海天气状况统计图\n\n## 其他月份\n\n### 风力分布分析\n\n2023年10月份上海风力等级和风向分布情况图\n\n省份历史天气查询\n\n黑龙江 吉林 辽宁 内蒙古 河北 山西 陕西 山东 新疆 西藏 青海 甘肃 宁夏 河南 江苏 湖北 浙江 安徽 福建 江西 湖南 贵州 四川 广东 云南 广西 海南 直辖市 北京 上海 天津 重庆 港澳台 香港 澳门 台湾\n\n上海2023年10月份天气统计概况\n\n月最高气温：27 ℃ (出现在2023-10-19) 月最低气温：14 ℃ (出现在2023-10-20) 【白天】晴天：12 天；下雨：4 天； 多云、阴天：15 天；下雪：0 天。 【夜间】晴天：15 天；下雨：3 天； 多云、阴天：13 天；下雪：0 天。  所有年份上海10月份天气统计\n\n上海城市区县历史天气查询 [...] | 2023年10月16日 | 晴 / 晴 | 25℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月17日 | 晴 / 晴 | 25℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月18日 | 晴 / 晴 | 26℃ / 19℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月19日 | 多云 / 小雨 | 27℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月20日 | 小雨 /

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain import hub
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)

template="""
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}

"""

# 自定义提示词模版
prompt_template = PromptTemplate.from_template(
    template=template
)

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", 
)

# 提供

# 获取Agent的实例: create_tool_calling
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)






> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "汕尾天气情况 2023年10月23日"
```[{'title': '23到25日平稳回温早晚清凉 - 广东- 中国天气网', 'url': 'http://gd.weather.com.cn/syxw/10/3661033.shtml', 'content': ':   曼谷 东京 首尔 吉隆坡 新加坡 巴黎 罗马 伦敦 雅典 柏林 纽约 温哥华 墨西哥城 哈瓦那 圣何塞 巴西利亚 布宜诺斯艾利斯 圣地亚哥 利马 基多 悉尼 墨尔本 惠灵顿 奥克兰 苏瓦 开罗 内罗毕 开普敦 维多利亚 拉巴特\n\n选择洲际\n\n:   亚洲 欧洲 北美洲 南美洲 非洲 大洋洲\n\n23到25日平稳回温 早晚清凉\n\n### 【字体：大;) 中;) 小;)】 2023-10-23 11:27:22   来源： 中国天气网广东站\n\n预计，23到25日我省大部多云为主，各地气温继续回升，早晚天气清凉，部分市县有（轻）雾。\n\n天气实况：22日我省大部阴天为主，出现了小雨，湛江西北部、茂名西部出现了局部中雨，湛江市廉江市塘莲镇录得单点的大雨25.5毫米；白天我省最高气温普遍上升，其中汕尾惠来录得全省最高气温28.6℃。\n\n具体预报：23日, 韶关、清远、肇庆、云浮晴间多云，粤东多云为主，其余市县多云间阴天，局部有零星小雨。\n\n24日，我省大部多云间晴天气为主，局部有零星小雨，其中受偏东气流影响，雷州半岛有中雨。最低气温：中北部市县18～21℃，南部市县21～23℃。', 'score': 0.89567816}, {'title': '汕尾2023年10月历史天气查询', 'url': 'https://www.zgjia.com/lishi/shanwei/202310.html', 'content': '汕尾2023年10月天气概述  广东省汕尾市2023年10月天气信息如下：整月白天平均温度为28.6度，夜间平均温度为23度，最高温度是发生在2023年10月05日的34度，最低温度是2023年10月08日的26摄氏度，本月天气总体以多云为

In [8]:
result = agent_exector.invoke({"input":"上海呢"})

print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "上海天气情况 2023年10月23日"
```[{'title': '上海历史天气预报2023年10月份', 'url': 'https://www.tianqihoubao.com/lishi/shanghai/month/202310.html', 'content': '前一月  后一月  上海历史天气  上海10月份天气统计信息 搜索"城市名+天气后报"访问本站（上海天气后报） 导出Excel\n\n### 天气状况统计\n\n2023年10月份上海上海天气状况统计图\n\n## 其他月份\n\n### 风力分布分析\n\n2023年10月份上海风力等级和风向分布情况图\n\n省份历史天气查询\n\n黑龙江 吉林 辽宁 内蒙古 河北 山西 陕西 山东 新疆 西藏 青海 甘肃 宁夏 河南 江苏 湖北 浙江 安徽 福建 江西 湖南 贵州 四川 广东 云南 广西 海南 直辖市 北京 上海 天津 重庆 港澳台 香港 澳门 台湾\n\n上海2023年10月份天气统计概况\n\n月最高气温：27 ℃ (出现在2023-10-19) 月最低气温：14 ℃ (出现在2023-10-20) 【白天】晴天：12 天；下雨：4 天； 多云、阴天：15 天；下雪：0 天。 【夜间】晴天：15 天；下雨：3 天； 多云、阴天：13 天；下雪：0 天。  所有年份上海10月份天气统计\n\n上海城市区县历史天气查询 [...] | 2023年10月16日 | 晴 / 晴 | 25℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月17日 | 晴 / 晴 | 25℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月18日 | 晴 / 晴 | 26℃ / 19℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月19日 | 多云 / 小雨 | 27℃ / 17℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月20日 | 小雨 /